## Wave equation
#### Parameter estimation for the Wave Equation using Gaussian processes (Without temporal discretization since we'd need a second order time scheme)


#### Problem Setup

$u_{tt} - c u_{xx} = 0$

General solution:
$u(x,t) = F(x-ct) + G(x+ct)$ with F, G some functions.

Take $F(x) = x^2$ and $G(x) = \sin(x)$ and $c=1$.

Thus: $u(x,t) = (x-t)^2 + \sin(x + t)$.

$u_0(x) := u(x,0) = x^2 + \sin(x)$

$x \in [0, 1], t \in [0,1]$

Set $f = 0$.

Consider $u$ to be a Gaussian process.

$u \sim \mathcal{GP}(0, k_{uu}(x_i, x_j, \theta))$

And the linear operator:

$\mathcal{L}_x^c = \frac{d^2}{dt^2} \cdot - c \frac{d^2}{dx^2} \cdot$

so that

$\mathcal{L}_x^c u = f$

Problem at hand: estimate $c$ (should be $c = 1$ in the end).


#### step 1: Simulate data

In [28]:
import time
import numpy as np
import sympy as sp
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [29]:
n = 10
np.random.seed(int(time.time()))
t = np.random.rand(n)
x = np.random.rand(n)

In [30]:
y_u = np.multiply(x-t, x-t) + np.sin(x+t)
y_f = 0*x

#### Step 2:Evaluate kernels

$k_{nn}(x_i, x_j; \theta) = \theta exp(-\frac{1}{2l_x}(x_i-x_j)^2 - \frac{1}{2l_t}(t_i-t_j)^2)$

In [31]:
x_i, x_j, t_i, t_j, theta, l_x, l_t, c = sp.symbols('x_i x_j t_i t_j theta l_x l_t c')
kuu_sym = theta*sp.exp(-1/(2*l_x)*((x_i - x_j)**2) - 1/(2*l_t)*((t_i - t_j)**2))
kuu_fn = sp.lambdify((x_i, x_j, t_i, t_j, theta, l_x, l_t), kuu_sym, "numpy")
def kuu(x, t, theta, l_x, l_t):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kuu_fn(x[i], x[j], t[i], t[j], theta, l_x, l_t)
    return k

$k_{ff}(x_i,x_j;\theta,\phi) \\
= \mathcal{L}_{\tilde{x}_i}^\nu \mathcal{L}_{\tilde{x}_j}^\nu k_{uu}(x_i, x_j; \theta) \\
= \frac{d^4}{dt_i^2 dt_j^2}k_{uu} - c\frac{d^4}{dt_i^2 dx_j^2}k_{uu} - c\frac{d^4}{dx_i^2 dt_j^2}k + c^2\frac{d^4}{dx_i^2 dx_j^2}k_{uu}$

In [32]:
kff_sym = sp.diff(kuu_sym, t_i, t_i, t_j, t_j) \
        - c*sp.diff(kuu_sym, t_i, t_i, x_j, x_j) \
        - c*sp.diff(kuu_sym, x_i, x_i, t_j, t_j) \
        + c**2*sp.diff(kuu_sym, x_i, x_i, x_j, x_j)
kff_fn = sp.lambdify((x_i, x_j, t_i, t_j, theta, l_x, l_t, c), kff_sym, "numpy")
def kff(x, t, theta, l_x, l_t, c):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kff_fn(x[i], x[j], t[i], t[j], theta, l_x, l_t, c)
    return k

$k_{fu}(x_i,x_j;\theta,\phi) \\
= \mathcal{L}_{\tilde{x}_i}^\nu k_{uu}(x_i, x_j; \theta) \\
= \frac{d^2}{dt_i^2}k_{uu} - c\frac{d^2}{dx_i^2}k_{uu}$

In [33]:
kfu_sym = sp.diff(kuu_sym, t_i, t_i) - c*sp.diff(kuu_sym, x_i, x_i)
kfu_fn = sp.lambdify((x_i, x_j, t_i, t_j, theta, l_x, l_t, c), kfu_sym, "numpy")
def kfu(x, t, theta, l_x, l_t, c):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kfu_fn(x[i], x[j], t[i], t[j], theta, l_x, l_t, c)
    return k

In [34]:
def kuf(x, t, theta, l_x, l_t, c):
    return kfu(x, t, theta, l_x, l_t, c).T

#### Step 3: Compute NLML

In [35]:
def nlml(params, x, t, y1, y2, s):
    theta_exp = np.exp(params[0]) 
    l_x_exp = np.exp(params[1])
    l_t_exp = np.exp(params[2]) # params[3] = c
    K = np.block([
        [kuu(x, t, theta_exp, l_x_exp, l_t_exp) + s*np.identity(x.size), kuf(x, t, theta_exp, l_x_exp, l_t_exp, params[3])],
        [kfu(x, t, theta_exp, l_x_exp, l_t_exp, params[3]), kff(x, t, theta_exp, l_x_exp, l_t_exp, params[3]) + s*np.identity(x.size)]
    ])
    y = np.concatenate((y1, y2))
    val = 0.5*(np.log(abs(np.linalg.det(K))) + np.mat(y) * np.linalg.inv(K) * np.mat(y).T)
    return val.item(0)

#### Step 4: Optimise hyperparameters

In [36]:
m = minimize(nlml, np.random.rand(4), args=(x, t, y_u, y_f, 1e-3), method="Nelder-Mead", options = {'maxiter' : 1000})

In [37]:
m.x[3]

0.0045685472526043065

In [38]:
m

 final_simplex: (array([[4.77154341e+00, 3.54645748e-01, 4.47177690e+00, 4.56854725e-03],
       [4.77146464e+00, 3.54633172e-01, 4.47172869e+00, 4.56896304e-03],
       [4.77155125e+00, 3.54653789e-01, 4.47178985e+00, 4.56921744e-03],
       [4.77161020e+00, 3.54701478e-01, 4.47178757e+00, 4.56884198e-03],
       [4.77161087e+00, 3.54678714e-01, 4.47178872e+00, 4.56891059e-03]]), array([-35.16159015, -35.16159015, -35.16159014, -35.16159014,
       -35.16159014]))
           fun: -35.16159014561473
       message: 'Optimization terminated successfully.'
          nfev: 342
           nit: 205
        status: 0
       success: True
             x: array([4.77154341e+00, 3.54645748e-01, 4.47177690e+00, 4.56854725e-03])